In [15]:
from audio_preprocessing.cut_audio_segment_from_audio_file import AudioProcessor
from feature_extraction.feature_extractor import FeatureExtractor
from feature_extraction.mfcc_images_extractor import MelSpectrogramImageExtractor
from data_processing.english_data_processing import ModelEvaluator
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [16]:
def extract_mfcc_images(n_mfcc, figsize, input_folder, output_folder):
    mfccImageExtractor = MelSpectrogramImageExtractor(n_mfcc, figsize)
    mfccImageExtractor.extract_from_folders(input_folder, output_folder)

In [17]:
def cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, input_folder, output_folder):
    segmentation = AudioProcessor(frame_size, hop_size, threshold)
    segmentation.process_folders(input_folder, output_folder)

In [18]:
def extract_feature(n_mfcc, input_folder, output_folder, selected_features):
    featureExtractor = FeatureExtractor(n_mfcc)
    featureExtractor.process_folder(input_folder, output_folder, selected_features)
    # featureExtractor.process_folder(input_folder, output_folder, ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

In [19]:
def model(featured_data):
    models = [RandomForestClassifier, ExtraTreesClassifier]
    
    df_train_test = pd.read_csv(featured_data)
    df_shuffled = df_train_test.sample(frac=1, random_state=42).reset_index(drop=True)

    modelEvaluator = ModelEvaluator(models)
    best_model, best_scaler, best_avg_acc = modelEvaluator.evaluate_models(df_shuffled)
    return best_model, best_scaler, best_avg_acc

In [20]:
def process_data(frame_size, hop_size, threshold, raw_folder, cutted_folder, n_mfcc_list, featured_data_path, n_mfcc_for_images, images_folder, features):
    cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, raw_folder, cutted_folder)
    n = len(features)

    results = []

    for n_mfcc in n_mfcc_list:
        for r in range(0, n + 1):
            for combo in combinations(range(n), r):
                selected_features = [features[i] for i in range(n) if i not in combo]
                extract_feature(n_mfcc, cutted_folder, featured_data_path, selected_features)
                current_model, current_scaler, current_avg_acc = model(featured_data_path)
                print(f"Scaler: {current_scaler}, Model: {current_model}, n_mfcc: {n_mfcc}, Selected features: {selected_features}, Accuracy: {current_avg_acc}")
                results.append({'Scaler': current_scaler, 'Model': current_model, 'N_MFCC': n_mfcc, 'Selected Features': selected_features, 'Average Accuracy': current_avg_acc})

    results_df = pd.DataFrame(results)
    ranked_results = results_df.sort_values(by='Average Accuracy', ascending=False)

    return ranked_results

In [21]:
process_data(256, 128, 0.005, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data', 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data1', [13,20,26,40], 'D:/data_analysis/speech_emotion_recognition/notebooks/feature3.csv', 100, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/images', ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

d:\data_analysis\speech_emotion_recognition\src\audio_preprocessing\utils\audio_utils.py:79: RuntimeWarning: invalid value encountered in divide
  return ste / np.max(ste)
2024-05-22 14:57:20.534 | INFO     | feature_extraction.feature_extractor:process_folder:148 - Processing input folder: D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data1
2024-05-22 15:00:13.179 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature3.csv
2024-05-22 15:00:13.324 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 15:01:46.424 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.622307241350838
2024-05-22 15:02:01.453 | INFO     | data_processing.english_data_processing:evaluate_models:74 

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc', 'mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6249448435306746


2024-05-22 15:11:27.743 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature3.csv
2024-05-22 15:11:27.832 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 15:12:39.313 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.620762612501032
2024-05-22 15:12:55.217 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6158552555528033
2024-05-22 15:14:03.683 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6188531087441169
2024-05-22 15:14:19.086 | INFO     | data_

Scaler: StandardScaler, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6256724465362068


2024-05-22 15:20:52.184 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature3.csv
2024-05-22 15:20:52.217 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 15:21:21.735 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5923115349682109
2024-05-22 15:21:30.882 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.5926749236231525
2024-05-22 15:22:00.797 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5934919494674264
2024-05-22 15:22:09.759 | INFO     | data

...

Scaler: MaxAbsScaler, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc_statistics'], Accuracy: 0.6168545949962844


2024-05-22 18:19:24.559 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature3.csv
2024-05-22 18:19:24.579 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 18:19:47.667 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5682209561555611
2024-05-22 18:19:55.401 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.5710388902650483
2024-05-22 18:20:18.495 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5700392205433077
2024-05-22 18:20:26.293 | INFO     | data

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 13, Selected features: ['mfcc'], Accuracy: 0.5717660804227562


2024-05-22 18:22:34.835 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature3.csv
2024-05-22 18:22:34.845 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....


ValueError: at least one array or dtype is required